# Parte 1: Estadísticas para ingenieros que miran el cielo


## 1.1 Promedio y desvio

### Melbourne

In [ ]:
import csv
archivo_melbourne='temperature_Melbourne_celsius.csv'
valores= []
promedio_melbourne = 0.0
with open(archivo_melbourne, 'r') as archivo:
    reader = csv.reader(archivo)
    next(reader)  
    valores = [float(fila[0]) for fila in reader]

promedio_melbourne = sum(valores) / len(valores)
varianza_melbourne = sum((x - promedio_melbourne)**2 for x in valores ) / (len(valores) - 1) 
desvio_melbourne=varianza_melbourne**-2

print(promedio_melbourne)
print(varianza_melbourne)
print(desvio_melbourne)


### Oslo

In [ ]:
import csv
archivo_melbourne='temperature_Oslo_celsius.csv'
valores= []
promedio_melbourne = 0.0
with open(archivo_melbourne, 'r') as archivo:
    reader = csv.reader(archivo)
    next(reader)  
    valores = [float(fila[0]) for fila in reader]

promedio_oslo = sum(valores) / len(valores)
varianza_oslo = sum((x - promedio_oslo)**2 for x in valores ) / (len(valores) - 1) 
desvio_oslo=varianza_oslo**-2

print(promedio_oslo)
print(varianza_oslo)
print(desvio_oslo)

### Quito

In [ ]:
import csv
archivo_melbourne='temperature_Quito_celsius.csv'
valores= []
promedio_melbourne = 0.0
with open(archivo_melbourne, 'r') as archivo:
    reader = csv.reader(archivo)
    next(reader)  
    valores = [float(fila[0]) for fila in reader]

promedio_quito = sum(valores) / len(valores)
varianza_quito = sum((x - promedio_quito)**2 for x in valores ) / (len(valores) - 1) 
desvio_quito=varianza_quito**-2

print(promedio_quito)
print(varianza_quito)
print(desvio_quito)